In [ ]:
from pathlib import Path
from eo_tides.utils import clip_models

In [ ]:
# traverse path and process files

input_dir = Path('/home/ubuntu/tide_models/')
output_dir = Path('/home/ubuntu/tide_models_indo')

clip_models(
    input_directory=input_dir,
    output_directory=output_dir,
    bbox=(94, -11, 142, 6)
)